In [1]:
import trainer.glue_base as glue_base
import models.sparse_token as sparse
import pickle, importlib
importlib.reload(glue_base)
importlib.reload(sparse)
Glue = glue_base.GlueAttentionApproxTrainer

/home/ainl/anaconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
subsets = ["cola"]#,"mnli","mrpc","qnli","qqp","rte","sst2","stsb","wnli",]
kss = [
    0.1, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 0.999, 'dynamic', 
    # 'dynamic:avg:avg:true', 'dynamic:avg:avg:false', 'dynamic:avg:max:true', 'dynamic:avg:max:false',
    # 'dynamic:max:avg:true', 'dynamic:max:avg:false', 'dynamic:max:max:true', 'dynamic:max:max:false',
]
sparse.benchmark_reset()
# subsets = ["mrpc"]
# kss = ['dynamic:avg:avg:f',0.1]

def get_score(score):
    if 'accuracy' in score:
        return score['accuracy'], "acc"
    first_metric = list(score.keys())[0]
    return score[first_metric], first_metric

results = {}
i = 0
for subset in subsets:
    trainer = Glue(dataset=subset, factor=16, batch_size=-1, device=0)
    trainer.load()
    scores = {}
    metric_name = ""
    bert_score, metric_name = get_score(trainer.eval_base_model())
    scores['bert'] = f'{bert_score:.5f}'
    for ks in kss:
        sparse.benchmark_reset()
        sparse_score, _ = get_score(trainer.eval_sparse_model(ks=ks))
        if isinstance(ks, str) and ks.startswith('dynamic'):
            est_k = sparse.benchmark_get_average('est_k')
            scores[str(ks)] = f'{sparse_score:.5f} (k:{est_k:.2f})'
        else:
            scores[str(ks)] = f'{sparse_score:.5f}'
        i += 1
        count = len(subsets) * len(kss)
        print(f'{i}/{count} | {subset} {ks} = {sparse_score:.5f}')
    results[f"{subset} ({metric_name})"] = scores

with open('glue_benchmark.pkl', 'wb') as f:
    pickle.dump(results, f)

sparse.benchmark_report()

Trainer: cola


Reusing dataset glue (/home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f0e04bff3297c11c.arrow
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-b9e7d91153cc59cf.arrow
Reusing dataset glue (/home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7e570c95fb4f22e4.arrow
Loading cached processed dataset at /home/ainl/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cac

Trainer: Save checkpoint path saves/glue-cola-16.pth


eval: 100%|██████████| 131/131 [00:03<00:00, 42.46it/s]


metric score {'matthews_correlation': 0.5338774230813111}
avg occupy 0.34522852946633237


...

metric score {'matthews_correlation': 0.5311983410233877}
avg occupy 0.34522852946633237
7/9 | cola 0.875 = 0.53120


eval: 100%|██████████| 131/131 [00:04<00:00, 32.26it/s]


metric score {'matthews_correlation': 0.5338774230813111}
avg occupy 0.34522852946633237
8/9 | cola 0.999 = 0.53388


eval: 100%|██████████| 131/131 [00:06<00:00, 21.28it/s]

metric score {'matthews_correlation': 0.5338774230813111}
avg occupy 0.34522852946633237
9/9 | cola dynamic = 0.53388
est_k: (1572, 762.2977443609105), avg: 0.48492222923722045


In [3]:
import pickle
import pandas as pd

with open('glue_benchmark.pkl', 'rb') as f:
    results = pickle.load(f)

data = []
subsets = list(results.keys())
factors = list(results[subsets[0]].keys())
for factor in factors:             
    row = []
    for subset in subsets:
        row.append(results[subset][factor])
    data.append(row)

#calculate reproducibility
data_scalar = []
for line in data:
    xs = []
    for item in line:
        xs.append(float(item.split()[0]))
    data_scalar.append(xs)
reproducibilities = []
for i in range(len(data_scalar)):
    rsum = 0
    for k in range(len(data_scalar[i])):
        rsum += data_scalar[i][k]/data_scalar[0][k]
    rsum /= len(data_scalar[i])
    reproducibilities.append(rsum)
for i, r in enumerate(reproducibilities):
    data[i].append(f"{r*100:.2f}")
subsets.append("reproduce")

df = pd.DataFrame(data, columns=subsets, index=factors)
tex = df.to_latex()
with open('saves_plot/glue_benchmark.tex', 'w') as f:
    f.write(tex)
df

/tmp/ipykernel_82072/3475623026.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tex = df.to_latex()


,cola (matthews_correlation),reproduce
bert,0.53388,100.00
0.1,0.05234,9.80
0.25,0.40688,76.21
0.375,0.46136,86.42
0.5,0.50255,94.13
0.625,0.52598,98.52
0.75,0.53117,99.49
0.875,0.53120,99.50
0.999,0.53388,100.00
dynamic,0.53388 (k:0.48),100.00


	cola (matthews_correlation)	mnli (acc)	mrpc (acc)	qnli (acc)	qqp (acc)	rte (acc)	sst2 (acc)	stsb (pearson)	wnli (acc)	reproduce
bert	0.53388	0.84198	0.84406	0.91543	0.90908	0.72563	0.92431	0.88047	0.56338	100.00
0.1	0.02214	0.57932	0.33507	0.59217	0.73391	0.52347	0.70872	0.01637	0.60563	57.36
0.25	0.33346	0.81559	0.36174	0.83892	0.89115	0.70758	0.85092	0.55826	0.56338	82.76
0.375	0.45301	0.83770	0.54377	0.89731	0.90616	0.70758	0.89335	0.75968	0.56338	91.88
0.5	0.49403	0.84167	0.76928	0.90976	0.90885	0.73285	0.90596	0.84956	0.56338	97.61
0.625	0.51112	0.84167	0.82957	0.91360	0.90915	0.72563	0.91628	0.87337	0.56338	99.12
0.75	0.52328	0.84198	0.84290	0.91433	0.90913	0.72924	0.92202	0.87936	0.56338	99.77
0.875	0.53120	0.84208	0.84348	0.91525	0.90913	0.72563	0.92317	0.88039	0.56338	99.92
0.999	0.53388	0.84198	0.84406	0.91543	0.90908	0.72563	0.92431	0.88047	0.56338	100.00

In [4]:
trainer = Glue(dataset='qnli', factor=16, batch_size=-1, device=0)
trainer.load() 

Trainer: qnli


KeyboardInterrupt: 

In [ ]:
trainer.model.bert = trainer.model_bert
trainer.eval_base_model()

eval: 100%|██████████| 683/683 [00:12<00:00, 54.43it/s]

metric score {'accuracy': 0.9154310818231741}
avg occupy 0.30243502260079336


{'accuracy': 0.9154310818231741}

In [ ]:
trainer.seed()
import models.sparse_token as sparse
import transformers.models.bert.modeling_bert as berts
import importlib
importlib.reload(sparse)

wrapped_bert = sparse.ApproxSparseBertModel(trainer.model_bert, approx_bert=trainer.approx_bert, ks=0.1)
sparse_cls_bert = berts.BertForSequenceClassification(trainer.model_bert.config)
sparse_cls_bert.load_state_dict(trainer.model.state_dict())
sparse_cls_bert.bert = wrapped_bert
sparse_cls_bert.to(trainer.device).eval()

trainer.eval_base_model(model = sparse_cls_bert)

eval: 100%|██████████| 683/683 [00:16<00:00, 42.48it/s]

metric score {'accuracy': 0.5921654768442248}
avg occupy 0.30243502260079336


{'accuracy': 0.5921654768442248}